In [1]:
library(rlang)
library(monocle3)
library(shiny)
library(ggplot2)
library(dplyr)
library(garnett)
library(Matrix)
library(irlba)
library(ComplexHeatmap)
library(ggplot2)
library(dplyr)
library(RColorBrewer)
library(circlize)

Loading required package: Biobase

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.



Attaching package: ‘Biobase’


The following object is masked from ‘package:rlang’:

    exprs


Loading required package: SingleCellExperiment

Loadin

# loading modulated genes and cds

In [2]:
modulated_genes=readRDS('/home/jovyan/scripts/renal_covid_19/scripts_for_publication/monocle_GO/allCD14_wave2_steroid_2021_modulated_genes.rds')

In [3]:
cds = readRDS('/home/jovyan/scripts/renal_covid_19/scripts_for_publication/monocle_GO/allCD14_wave2_steroid_2021_cds.rds')

In [4]:
unique(cds$individual_id)

[1] C33  C101 C146 C138 C145 C147
Levels: C101 C138 C145 C146 C147 C33

# selected sig genes

In [5]:
genes <- row.names(subset(modulated_genes, q_value <0.05 & morans_I > 0.2 ))#& morans_I > 0.2
length(genes)

[1] 102

In [6]:
# remove mito and ribo genes
genes <-genes[!grepl(paste0('^RPL', collapse = "|"), genes)]
genes <-genes[!grepl(paste0('^RPS', collapse = "|"), genes)]
genes <-genes[!grepl(paste0('^MT-', collapse = "|"), genes)]

length(genes)

[1] 58

# get count matrix for sig genes in pseudo time order

In [7]:
unique((cds$days_from_steroid))

[1]   6  49  34   5  13  -6   1  14 -10  11   2  -3   4  -1   9   7  -2   0  -4
[20]  18  -8  -5

In [8]:
pt.matrix <- as.matrix(cds@assays@data$counts[match(genes,rownames(rowData(cds))),order(cds$days_from_steroid)]) # order by pseudotime_GPLVM

In [9]:
cellname <-colnames(pt.matrix)

# smoothening

In [10]:
pt.matrix <- t(apply(pt.matrix,1,function(x){smooth.spline(x,df=3)$y}))
pt.matrix <- t(apply(pt.matrix,1,function(x){(x-mean(x))/sd(x)}))

In [11]:
rownames(pt.matrix) <- genes;

In [12]:
colnames(pt.matrix) <- cellname;

# getting cluster bars

In [13]:
unique(cds$WHO_temp_severity) #checking clusters

[1] critical nan      mild     severe   moderate
Levels: critical mild moderate nan severe

In [14]:
unique(cds$case_control)

[1] POSITIVE RECOVERY
Levels: POSITIVE RECOVERY

In [15]:
cds$WHO_temp_severity_correct <- factor(cds$WHO_temp_severity, levels = c('nan','mild', 'moderate','severe','critical'), labels = c('recovery','mild','moderate','severe','critical'))

In [16]:
unique(cds$WHO_temp_severity_correct)

[1] critical recovery mild     severe   moderate
Levels: recovery mild moderate severe critical

In [17]:
pseudo_time <-pseudotime(cds)
pseudo_time <- as.data.frame(pseudo_time)
#print (head(pseudo_time))

In [18]:
day_from_steroid <-(colData(cds)[, "days_from_steroid"])
day_from_steroid <- as.data.frame(day_from_steroid)
#tail(day_from_steroid)

In [19]:
pseudotime_GPLVM <-(colData(cds)[, "pseudotime_GPLVM"])
pseudotime_GPLVM <- as.data.frame(pseudotime_GPLVM)
#tail(pseudotime_GPLVM)

In [20]:
sev <-(colData(cds)[, "WHO_temp_severity_correct"])
df_sev <- as.data.frame(sev)
#tail(df_sev)

In [21]:
positive_recovery <-(colData(cds)[, "case_control"])
positive_recovery <- as.data.frame(positive_recovery)
#tail(case_control)

In [22]:
time_from_first_symptoms<-(colData(cds)[, "time_from_first_symptoms"])
time_from_first_symptoms <- as.data.frame(time_from_first_symptoms)
#tail(time_from_first_symptoms)

In [23]:
time_from_pos_swab<-(colData(cds)[, "time_from_positive_swab"])
time_from_pos_swab <- as.data.frame(time_from_pos_swab)
#tail(time_from_pos_swab)

In [24]:
time_from_inf <-(colData(cds)[, "time_from_infection"])
time_from_inf <- as.data.frame(time_from_inf)
#tail(time_from_inf)

In [25]:
top_anno <- cbind(positive_recovery,time_from_inf,pseudotime_GPLVM,df_sev,pseudo_time,time_from_pos_swab,time_from_first_symptoms,day_from_steroid)

In [26]:
top_anno <-top_anno[order(top_anno$day_from_steroid),]
head(top_anno)

,positive_recovery,time_from_inf,pseudotime_GPLVM,sev,pseudo_time,time_from_pos_swab,time_from_first_symptoms,day_from_steroid
,<fct>,<int>,<dbl>,<fct>,<dbl>,<int>,<int>,<int>
UK-CIC10690336+UK-CIC10690528_TTTGGTTAGCGTGTCC-1,POSITIVE,3,0.3541150,mild,13.6374608,3,-9,-10
UK-CIC10690333+UK-CIC10690525_GGACGTCTCAACACTG-1,POSITIVE,3,0.4538487,mild,13.6374614,3,-9,-10
UK-CIC10690334+UK-CIC10690526_GGACGTCTCGACAGCC-1,POSITIVE,3,0.4898367,mild,0.5490350,3,-9,-10
UK-CIC10690335+UK-CIC10690527_GAAACTCTCGATGAGG-1,POSITIVE,3,0.4843546,mild,0.5490353,3,-9,-10
UK-CIC10690335+UK-CIC10690527_TCGTACCTCAACACCA-1,POSITIVE,3,0.3387644,mild,0.5490356,3,-9,-10
UK-CIC10690336+UK-CIC10690528_AGGCCACAGCGATAGC-1,POSITIVE,3,0.5235588,mild,1.8175063,3,-9,-10


# get colann and plot heatmap

In [ ]:
library(ComplexHeatmap)
library(ggplot2)
library(dplyr)
library(RColorBrewer)
library(circlize)

ann <- data.frame(top_anno$sev,top_anno$pseudotime_GPLVM, top_anno$positive_recovery,top_anno$time_from_inf,top_anno$pseudo_time,top_anno$time_from_pos_swab,top_anno$time_from_first_symptoms,top_anno$day_from_steroid)#top_anno$day_from_steroid,
colnames(ann) <- c('sev','pseudotime_GPLVM', 'positive_recovery','time_from_inf','pseudotime','time_from_pos_swab','time_from_first_symptoms','day_from_steroid')#,
colours <- list('sev' = c('recovery'='green','mild' = 'gold','moderate' ='red','severe' = 'royalblue','critical'='black'),            
  'positive_recovery' = c('POSITIVE' = 'limegreen','RECOVERY'='purple'),#, 'RECOVERY' = 'gold'
    'time_from_inf' = colorRamp2(c(min(top_anno$time_from_inf), max(top_anno$time_from_inf)), c("white", "blue")),
                'day_from_steroid' = colorRamp2(c(min(top_anno$day_from_steroid), max(top_anno$day_from_steroid)), c("white", "red")),
                'time_from_pos_swab' = colorRamp2(c(min(top_anno$time_from_pos_swab), max(top_anno$time_from_pos_swab)), c("white", "green")),
                'time_from_first_symptoms' = colorRamp2(c(min(top_anno$time_from_first_symptoms), max(top_anno$time_from_first_symptoms)), c("white", "black"))
        )
colAnn <- HeatmapAnnotation(df = ann,
  which = 'col',
  col = colours,
  annotation_width = unit(c(1, 4), 'cm'),
  gap = unit(1, 'mm'))

In [ ]:
pdf("/home/jovyan/scripts/renal_covid_19/steroid_pipeline_corrected/trajectory_with_actual_timeline/heatmap_actual_steroid_timeline.pdf", 
    height=15, width=10)
hthc <- ComplexHeatmap::Heatmap(
pt.matrix,
  name                         = "z-score",
  col                          = colorRamp2(seq(from=-2,to=2,length=11),rev(brewer.pal(11, "Spectral"))),
  show_row_names               = FALSE,
  show_column_names            = FALSE,
  row_names_gp                 = gpar(fontsize = 12),
  clustering_method_rows = "ward.D2",
  clustering_method_columns = "ward.D2",
  row_title_rot                = 0,
  cluster_rows                 = TRUE,
  cluster_row_slices           = FALSE,
  cluster_columns              = FALSE,
  width = unit(12.8, "cm"), height = unit(25, "cm"),
  top_annotation = colAnn)
 # bottom_annotation = ba)
print(hthc)


#file =paste0("/home/jovyan/scripts/renal_covid_19/steroid_pipeline_corrected/steroid_pipeline/trajectory_with_Rik_model_output/")
#dev.copy(pdf,filename=paste0(file,'allCD14_wave2_steroid_2021_norow_test.pdf'),width=1200, height=2500);
dev.off ();

In [ ]:
#write.csv(x=top_anno, file="/home/jovyan/scripts/renal_covid_19/steroid_pipeline/trajectory_with_Rik_model_output/ordered_cells.csv")

In [ ]:
#write.csv(x=pt.matrix, file="/home/jovyan/scripts/renal_covid_19/steroid_pipeline/trajectory_with_Rik_model_output/genes_along_trajectory_pseudotimeGPLVM.csv")

In [27]:
library(ComplexHeatmap)
library(ggplot2)
library(dplyr)
library(RColorBrewer)
library(circlize)

ann <- data.frame(top_anno$positive_recovery,top_anno$day_from_steroid)#top_anno$day_from_steroid,
colnames(ann) <- c('positive_recovery','day_from_steroid')#,
colours <- list(            
'positive_recovery' = c('POSITIVE' = 'limegreen','RECOVERY'='purple'),#, 'RECOVERY' = 'gold'
'day_from_steroid' = colorRamp2(c(min(top_anno$day_from_steroid), max(top_anno$day_from_steroid)), c("gray", "red"))
)
colAnn <- HeatmapAnnotation(df = ann,
  which = 'col',
  col = colours,
  annotation_width = unit(c(1, 4), 'cm'),
  gap = unit(1, 'mm'))

In [28]:
pdf("/home/jovyan/scripts/renal_covid_19/scripts_for_publication/monocle_GO/heatmap_actual_steroid_timeline1.pdf", 
    height=15, width=10,)
hthc <- ComplexHeatmap::Heatmap(
pt.matrix,
  name                         = "z-score",
  col                          = colorRamp2(seq(from=-2,to=2,length=11),rev(brewer.pal(11, "Spectral"))),
  show_row_names               = TRUE,
  show_column_names            = FALSE,
  row_names_gp                 = gpar(fontsize = 12),
  clustering_method_rows = "ward.D2",
  clustering_method_columns = "ward.D2",
  row_title_rot                = 0,
  cluster_rows                 = TRUE,
  cluster_row_slices           = FALSE,
  cluster_columns              = FALSE,
  width = unit(12.8, "cm"), height = unit(25, "cm"),
  top_annotation = colAnn)
 # bottom_annotation = ba)

print(hthc)

dev.off ();

`use_raster` is automatically set to TRUE for a matrix with more than
2000 columns You can control `use_raster` argument by explicitly
setting TRUE/FALSE to it.

Set `ht_opt$message = FALSE` to turn off this message.

'magick' package is suggested to install to give better rasterization.

Set `ht_opt$message = FALSE` to turn off this message.



png 
  2

In [29]:
unique(top_anno$time_from_inf)

[1]  3  5  2  7  4 10 12 14  9 17 18 11 19 16 47 54

In [30]:
unique(top_anno$time_from_pos_swab)

[1]  3  5  2  7  4 10 12 14  9 17 18 11 19 16 47 54

In [31]:
unique(top_anno$time_from_first_symptoms)

[1] -9 -7  0 -5  2 -2  5  7  9 12  6 14 18 35 50

In [32]:
unique(top_anno$day_from_steroid)

[1] -10  -8  -6  -5  -4  -3  -2  -1   0   1   2   4   5   6   7   9  11  13  14
[20]  18  34  49